In [1]:
import librosa
import tensorflow as tf
import numpy as np


In [2]:
file_name = "../Referencni_skladby/Imanbek & BYOR - Belly Dancer (Official Music Video).wav"

In [3]:
y, sr = librosa.load(file_name,sr=22050, mono=True, duration=25)

In [4]:
def get_melspec(y, sr):
    n_mels = 128
    hop_length = 512
    n_fft = 1024
    #calculate the melspectogram of the provided audio wave
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    
    return S

In [5]:
mel = get_melspec(y,sr)

In [6]:
vstupni_tensor = tf.constant(mel)

In [7]:
vstupni_tensor_reshaped = tf.expand_dims(vstupni_tensor, axis=0)  # Přidává "batch size" dimenzi
vstupni_tensor_reshaped = tf.expand_dims(vstupni_tensor_reshaped, axis=-1)  # Přidává "channels" dimenzi

In [8]:
saved_model_dir = 'model_muj_savedmodel_googleColab_trained/'

# Načtení modelu
model = tf.saved_model.load(saved_model_dir)
model_infer = model.signatures["serving_default"]

In [9]:
vysledek = model_infer(vstupni_tensor_reshaped)
vysledek

{'dense_3': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[3.2003166e-03, 6.1732206e-13, 9.8057101e-12, 3.2357400e-04,
         9.9647611e-01, 3.4047146e-15, 3.9812550e-17, 1.4003349e-13,
         7.2330482e-20, 5.1772083e-16]], dtype=float32)>}

In [10]:
predictions_tensor = vysledek['dense_3']

In [11]:
predictions = predictions_tensor.numpy()

In [15]:
genres = {0: 'blues', 1 : 'classical', 2: 'country', 3 : 'disco', 4 : 'hiphop',  5 : 'jazz' , 6 : 'metal', 7 : 'pop', 8 : 'reggae', 9 : 'rock'}

In [18]:
labels_list = [genres[i] for i in range(10)]

In [19]:
# Vytvoření pole pro labely
labels_array = np.array(labels_list).reshape(-1, 1)  # Přeformátování na tvar (10, 1)

# Přeformátování výstupních dat, pokud je třeba
predictions = predictions.reshape(-1, 1)  # Ujistěte se, že má tvar (10, 1)

# Sloučení labelů a výstupních dat
final_array = np.hstack((labels_array, predictions))  # Sloučení do pole (10, 2)

print(final_array)

[['blues' '0.0032003166']
 ['classical' '6.1732206e-13']
 ['country' '9.80571e-12']
 ['disco' '0.000323574']
 ['hiphop' '0.9964761']
 ['jazz' '3.4047146e-15']
 ['metal' '3.981255e-17']
 ['pop' '1.4003349e-13']
 ['reggae' '7.233048e-20']
 ['rock' '5.1772083e-16']]
